In [1]:
# Generate Random Latitudes and Longitudes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
# add citipy library
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)


lat_lngs = zip(lats, lngs)
lat_lngs


# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []

# Identify nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, add it to the cities list.
    if city not in cities:
        cities.append(city)
        
len(cities)

10

In [4]:
# Import the request library.
import requests

# Import API Key from config.py file
from config import weather_api_key
# print (weather_api_key)

# Import date time to recast api date a datetime format
import time
from datetime import datetime

In [5]:
# Starting URL for Weather Map API Call.
url = f"http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID={weather_api_key}"
# print(url)

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # print(requests.get(city_url).json())
    
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | chervishevo
Processing Record 2 of Set 1 | littlehampton
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | camapua
Processing Record 5 of Set 1 | east liverpool
Processing Record 6 of Set 1 | new norfolk
Processing Record 7 of Set 1 | ribeira grande
Processing Record 8 of Set 1 | kavaratti
Processing Record 9 of Set 1 | hilo
Processing Record 10 of Set 1 | broken hill
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chervishevo,56.9464,65.4278,23.56,95,100,7.05,RU,2021-12-19 06:03:18
1,Littlehampton,50.8069,-0.5378,45.99,89,100,12.50,GB,2021-12-19 06:03:18
2,Punta Arenas,-53.1500,-70.9167,51.12,71,89,20.40,CL,2021-12-19 06:02:31
3,Camapua,-19.5314,-54.0439,72.48,93,100,3.65,BR,2021-12-19 06:03:19
4,East Liverpool,40.6187,-80.5773,38.25,95,100,8.01,US,2021-12-19 06:03:19
5,New Norfolk,-42.7826,147.0587,63.90,48,99,7.00,AU,2021-12-19 06:03:20
6,Ribeira Grande,38.5167,-28.7000,60.84,71,100,29.46,PT,2021-12-19 06:03:20
7,Kavaratti,10.5669,72.6420,82.18,63,11,7.72,IN,2021-12-19 06:03:20
8,Hilo,19.7297,-155.0900,73.27,46,90,5.75,US,2021-12-19 06:03:21
9,Broken Hill,-31.9500,141.4333,95.07,14,60,22.53,AU,2021-12-19 06:03:21


In [8]:
# reorder the columns
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 
                    'Cloudiness', 'Wind Speed']

# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Chervishevo,RU,56.9464,65.4278,23.56,95,100,7.05
1,Littlehampton,GB,50.8069,-0.5378,45.99,89,100,12.50
2,Punta Arenas,CL,-53.1500,-70.9167,51.12,71,89,20.40
3,Camapua,BR,-19.5314,-54.0439,72.48,93,100,3.65
4,East Liverpool,US,40.6187,-80.5773,38.25,95,100,8.01
5,New Norfolk,AU,-42.7826,147.0587,63.90,48,99,7.00
6,Ribeira Grande,PT,38.5167,-28.7000,60.84,71,100,29.46
7,Kavaratti,IN,10.5669,72.6420,82.18,63,11,7.72
8,Hilo,US,19.7297,-155.0900,73.27,46,90,5.75
9,Broken Hill,AU,-31.9500,141.4333,95.07,14,60,22.53


In [10]:
# Create the output file (CSV).
output_data_file = "weather_database/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")